<a href="https://colab.research.google.com/github/eduardocarbo/NewRepo/blob/master/04_SVM_para_regresi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

In [4]:
bd = pd.read_csv("USA_Housing.csv")

In [6]:
del bd["Address"]

In [7]:
x = bd[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population']]
y = bd[['Price']]

In [8]:
x.describe()
x.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.458574,5.682861,7.009188,4.09,23086.800503
1,79248.642455,6.002900,6.730821,3.09,40173.072174
2,61287.067179,5.865890,8.512727,5.13,36882.159400
3,63345.240046,7.188236,5.586729,3.26,34310.242831
4,59982.197226,5.040555,7.839388,4.23,26354.109472


In [9]:
#estandarización de los datos
scl_x = StandardScaler()
scl_y = StandardScaler()
X = scl_x.fit_transform(x) 
Y = scl_y.fit_transform(y) 

In [10]:
#Selección de subconjuntos de train(.7) y test(.3)
#TIP: presionar shift + tab para ver como se define una función 
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.7, test_size=0.3)

In [12]:
svr = SVR(kernel='linear',C=0.7,epsilon=0.15)

In [16]:
modelo = svr.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
y_pred = svr.predict(X_test)

In [19]:
precision = svr.score(X_test,y_test)
precision

0.9153029066794112

In [20]:
#abs_dif = np.abs(y_pred - y_test)/y_test
#mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
#mape

In [21]:
mape = mean_absolute_error(y_test, y_pred)*100
print(mape)

23.1715440006106


In [22]:
########### TUNNING DEL MODELO ########
svr = SVR()
kernel = ["linear", "rbf"]
#tolerance = [0.001, 0.01,0.005]
tolerance = [0.001, 0.005,0.002]
C = [0.5,0.35]
grid = dict(kernel=kernel, tol=tolerance, C=C)

In [23]:
grid

{'C': [0.5, 0.35], 'kernel': ['linear', 'rbf'], 'tol': [0.001, 0.005, 0.002]}

In [24]:
# initialize a cross-validation fold and perform a grid-search to
# tune the hyperparameters
print("[INFO] grid searching over the hyperparameters...")

cvFold = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

gridSearch = GridSearchCV(estimator=svr, param_grid=grid, n_jobs=-1,
                         cv=cvFold, scoring="neg_mean_squared_error")

searchResults = gridSearch.fit(X_train, y_train)

# extract the best model and evaluate it
print("[INFO] evaluating...")
bestModel = searchResults.best_estimator_
print("R2: {:.2f}".format(bestModel.score(X_test, y_test)))

[INFO] grid searching over the hyperparameters...


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[INFO] evaluating...
R2: 0.92


In [ ]:
gridSearch.best_params_

{'C': 0.35, 'kernel': 'linear', 'tol': 0.002}